In [1]:
import urllib3
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler

import os
import sys
import logging
from tqdm import tqdm
import numpy as np
from datetime import datetime
from filelock import FileLock
from prettytable import PrettyTable
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union

from transformers import (
    BertTokenizer,
    BertPreTrainedModel,
    BertModel,
    BertConfig
)
from transformers.file_utils import (
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    add_code_sample_docstrings,
    add_code_sample_docstrings
)
from transformers.modeling_outputs import (
    SequenceClassifierOutput
)
from transformers.trainer import Trainer,TrainingArguments
from datasets import Dataset, load_dataset, load_from_disk

logger = logging.getLogger(__name__)

C:\Users\fooli\anaconda3\envs\gpu_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\fooli\anaconda3\envs\gpu_env\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\fooli\anaconda3\envs\gpu_env\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\fooli\anaconda3\envs\gpu_env\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node inst

In [2]:

# Set path to SentEval
PATH_TO_SENTEVAL = os.getcwd()+'/SentEval-main'
PATH_TO_DATA =  os.getcwd()+'/SentEval-main/data'

# Import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
batch_size=64
train_dataset = load_from_disk("wiki_for_sts_32")
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last = True)

In [5]:
# evaluate model in all STS tasks
def print_table(task_names, scores):
    tb = PrettyTable()
    tb.field_names = task_names
    tb.add_row(scores)
    print(tb)
    
def evalModel(model,tokenizer, pooler): 
    tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    
    params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
    params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                'tenacity': 3, 'epoch_size': 2}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    def prepare(params, samples):
        return

    def batcher(params, batch, max_length=None):
            # Handle rare token encoding issues in the dataset
            if len(batch) >= 1 and len(batch[0]) >= 1 and isinstance(batch[0][0], bytes):
                batch = [[word.decode('utf-8') for word in s] for s in batch]

            sentences = [' '.join(s) for s in batch]
            
            batch = tokenizer.batch_encode_plus(
                sentences,
                return_tensors='pt',
                padding=True,
                max_length=max_length,
                truncation=True
            )
            # Move to the correct device
            for k in batch:
                batch[k] = batch[k].to(device)
            
            # Get raw embeddings
            with torch.no_grad():
                pooler_output = model(**batch, output_hidden_states=True, return_dict=True)
                if pooler == "cls_before_pooler":
                    pooler_output = pooler_output.last_hidden_state[:, 0]
                elif pooler == "cls_after_pooler":
                    pooler_output = pooler_output.pooler_output

            return pooler_output.cpu()
    results = {}

    for task in tasks:
        se = senteval.engine.SE(params, batcher, prepare)
        result = se.eval(task)
        results[task] = result
    task_names = []
    scores = []
    for task in ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']:
        task_names.append(task)
        if task in results:
            if task in ['STS12', 'STS13', 'STS14', 'STS15', 'STS16']:
                #print(f"task {task} results[task] {results[task]}")
                scores.append("%.2f" % (results[task]['all']['spearman']['mean'] * 100))
            else:
                print(f"task {task} results[task] {results[task]}")
                scores.append("%.2f" % (results[task]['spearman'] * 100))
        else:
            scores.append("0.00")
    task_names.append("Avg.")
    scores.append("%.2f" % (sum([float(score) for score in scores]) / len(scores)))
    print_table(task_names, scores)

    return sum([float(score) for score in scores])/len(scores)

In [6]:
class MLPLayer(nn.Module):
    """
    Head for getting sentence representations over RoBERTa/BERT's CLS representation.
    """
    def __init__(self, hidden_size):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.Tanh()

    def forward(self, features, **kwargs):
        x = self.dense(features)
        x = self.activation(x)

        return x
    
class Similarity(nn.Module):
    """
    Dot product or cosine similarity
    """

    def __init__(self, temp):
        super().__init__()
        self.temp = temp
        self.cos = nn.CosineSimilarity(dim=-1)

    def forward(self, x, y):
        return self.cos(x, y) / self.temp

In [7]:
config = BertConfig()

In [8]:
# doc text needed for transformers model

_CHECKPOINT_FOR_DOC = "bert-base-uncased"
_CONFIG_FOR_DOC = "BertConfig"
_TOKENIZER_FOR_DOC = "BertTokenizer"


BERT_START_DOCSTRING = r"""

    This model inherits from :class:`~transformers.PreTrainedModel`. Check the superclass documentation for the generic
    methods the library implements for all its model (such as downloading or saving, resizing the input embeddings,
    pruning heads etc.)

    This model is also a PyTorch `torch.nn.Module <https://pytorch.org/docs/stable/nn.html#torch.nn.Module>`__
    subclass. Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to
    general usage and behavior.
headlines
    Parameters:
        config (:class:`~transformers.BertConfig`): Model configuration class with all the parameters of the model.
            Initializing with a config file does not load the weights associated with the model, only the
            configuration. Check out the :meth:`~transformers.PreTrainedModel.from_pretrained` method to load the model
            weights.
"""

BERT_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (:obj:`torch.LongTensor` of shape :obj:`({0})`):
            Indices of input sequence tokens in the vocabulary.

            Indices can be obtained using :class:`~transformers.BertTokenizer`. See
            :meth:`transformers.PreTrainedTokenizer.encode` and :meth:`transformers.PreTrainedTokenizer.__call__` for
            details.

            `What are input IDs? <../glossary.html#input-ids>`__
        attention_mask (:obj:`torch.FloatTensor` of shape :obj:`({0})`, `optional`):
            Mask to avoid performing attention on padding token indices. Mask values selected in ``[0, 1]``:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.

            `What are attention masks? <../glossary.html#attention-mask>`__
        token_type_ids (:obj:`torch.LongTensor` of shape :obj:`({0})`, `optional`):
            Segment token indices to indicate first and second portions of the inputs. Indices are selected in ``[0,
            1]``:

            - 0 corresponds to a `sentence A` token,
            - 1 corresponds to a `sentence B` token.

            `What are token type IDs? <../glossary.html#token-type-ids>`_
        position_ids (:obj:`torch.LongTensor` of shape :obj:`({0})`, `optional`):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range ``[0,
            config.max_position_embeddings - 1]``.

            `What are position IDs? <../glossary.html#position-ids>`_
        head_mask (:obj:`torch.FloatTensor` of shape :obj:`(num_heads,)` or :obj:`(num_layers, num_heads)`, `optional`):
            Mask to nullify selected heads of the self-attention modules. Mask values selected in ``[0, 1]``:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        inputs_embeds (:obj:`torch.FloatTensor` of shape :obj:`({0}, hidden_size)`, `optional`):
            Optionally, instead of passing :obj:`input_ids` you can choose to directly pass an embedded representation.
            This is useful if you want more control over how to convert :obj:`input_ids` indices into associated
            vectors than the model's internal embedding lookup matrix.
        output_attentions (:obj:`bool`, `optional`):
            Whether or not to return the attentions tensors of all attention layers. See ``attentions`` under returned
            tensors for more detail.
        output_hidden_states (:obj:`bool`, `optional`):
            Whether or not to return the hidden states of all layers. See ``hidden_states`` under returned tensors for
            more detail.
        return_dict (:obj:`bool`, `optional`):
            Whether or not to return a :class:`~transformers.file_utils.ModelOutput` instead of a plain tuple.
"""

In [9]:
# implement of SimCSEModel

@add_start_docstrings(
    """
    Bert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled
    output) e.g. for GLUE tasks.
    """,
    BERT_START_DOCSTRING,
)
class SimCSEModel(BertPreTrainedModel):
    def __init__(self, config,temp=0.05):
        super().__init__(config)

        self.bert = BertModel(config)
        self.mlp = MLPLayer(config.hidden_size)
        self.sim = Similarity(temp)
        
        self.init_weights()

    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    @add_code_sample_docstrings(
        processor_class=_TOKENIZER_FOR_DOC,
        checkpoint=_CHECKPOINT_FOR_DOC,
        output_type=SequenceClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
    )
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        sent_emb=False, # if true, return sentence embedding for evaluation
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        if sent_emb:
            # return sentence embedding for evaluation
            
            outputs = self.bert(
                input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )

            return outputs
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        # view_0 & view_1 are the same sentence go through bert twice
        outputs_view_0 = self.bert(
            input_ids[:,0],
            attention_mask=attention_mask[:,0],
            token_type_ids=token_type_ids[:,0],
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        outputs_view_1 = self.bert(
            input_ids[:,1],
            attention_mask=attention_mask[:,1],
            token_type_ids=token_type_ids[:,1],
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        # go through mlp twice
        pooled_output_0 = self.mlp(outputs_view_0.last_hidden_state[:, 0])
        pooled_output_1 = self.mlp(outputs_view_1.last_hidden_state[:, 0])

        sim_matrix = self.sim(pooled_output_0.unsqueeze(1), pooled_output_1.unsqueeze(0))
        
        # labels = [0,1,...,batch_size-1]
        labels = torch.arange(0,sim_matrix.shape[0],step=1).cuda()
        
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(sim_matrix,labels)
        
        return SequenceClassifierOutput(
            loss=loss,
            logits=sim_matrix,
            hidden_states=[pooled_output_0,pooled_output_1],
            attentions=None,
        )

In [10]:
# override the evaluate method
class SimCSETrainer(Trainer):
    def __init__(self,**paraments):
        super().__init__(**paraments)
        
        self.best_sts = 0.0
        
    def evaluate(
        self,
        eval_dataset: Optional[Dataset] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
        eval_senteval_transfer: bool = False,
    ) -> Dict[str, float]:

        # SentEval prepare and batcher
        def prepare(params, samples):
            return

        def batcher(params, batch):
            sentences = [' '.join(s) for s in batch]
            batch = self.tokenizer.batch_encode_plus(
                sentences,
                return_tensors='pt',
                padding=True,
            )
            for k in batch:
                batch[k] = batch[k].to(self.args.device)
            with torch.no_grad():
                outputs = self.model(**batch, output_hidden_states=True, return_dict=True, sent_emb=True)
                pooler_output = outputs.last_hidden_state[:, 0]
            return pooler_output.cpu()

        # Set params for SentEval (fastmode)
        params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
        params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                            'tenacity': 3, 'epoch_size': 2}

        se = senteval.engine.SE(params, batcher, prepare)
        tasks = ['STSBenchmark', 'SICKRelatedness']
        self.model.eval()
        results = se.eval(tasks)
        #print( results['STSBenchmark'] )
        stsb_spearman = results['STSBenchmark']['spearman']
        sickr_spearman = results['SICKRelatedness']['spearman']

        metrics = {"eval_stsb_spearman": stsb_spearman, "eval_sickr_spearman": sickr_spearman, "eval_avg_sts": (stsb_spearman + sickr_spearman) / 2} 
        print(metrics)
        
        # save and eval model
        if metrics["eval_avg_sts"]>self.best_sts:
            self.best_sts = metrics["eval_avg_sts"]
            evalModel(self.model.bert,tokenizer, pooler = 'cls_before_pooler')
            self.save_model(self.args.output_dir+"/best-model")
            
        self.log(metrics)
        return metrics

In [11]:
model = SimCSEModel(config).from_pretrained("bert-base-uncased").cuda()

Some weights of SimCSEModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['mlp.dense.bias', 'mlp.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
args = TrainingArguments(
    output_dir = 'trainer_models',
    evaluation_strategy   = "steps",
    eval_steps            = 125,
    learning_rate         = 3e-5,
    num_train_epochs      = 1.0,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size  = 64,
)

In [13]:
#import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.3.1+cu121
Is CUDA enabled? True


In [14]:
trainer = SimCSETrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

C:\Users\fooli\anaconda3\envs\gpu_env\lib\site-packages\accelerate\accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [15]:
# test the evaluate method and see what are the initial results
trainer.evaluate()

{'eval_stsb_spearman': 0.5037055626042174, 'eval_sickr_spearman': 0.6510762547103278, 'eval_avg_sts': 0.5773909086572726}
MSRpar
MSRvid
SMTeuroparl
surprise.OnWN
surprise.SMTnews


C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


FNWN
headlines
OnWN
deft-forum
deft-news
headlines
images
OnWN
tweet-news
answers-forums
answers-students
belief
headlines
images
answer-answer
headlines
plagiarism
postediting
question-question
task STSBenchmark results[task] {'devpearson': 0.5759484546752985, 'pearson': 0.5000071366357604, 'spearman': 0.5037055626042174, 'mse': 1.9113272058490889, 'yhat': array([2.53563102, 2.34177347, 3.50484297, ..., 2.26899934, 3.69027919,
       4.39055101]), 'ndev': 1500, 'ntest': 1379}
task SICKRelatedness results[task] {'devpearson': 0.7027961395366927, 'pearson': 0.7018062321019634, 'spearman': 0.6510762547103278, 'mse': 0.5183835427621566, 'yhat': array([3.28330554, 4.05144345, 2.8199942 , ..., 2.16892583, 3.15235038,
       4.38313658]), 'ndev': 500, 'ntest': 4927}
+-------+-------+-------+-------+-------+--------------+-----------------+-------+
| STS12 | STS13 | STS14 | STS15 | STS16 | STSBenchmark | SICKRelatedness |  Avg. |
+-------+-------+-------+-------+-------+--------------+-------

{'eval_stsb_spearman': 0.5037055626042174,
 'eval_sickr_spearman': 0.6510762547103278,
 'eval_avg_sts': 0.5773909086572726}

In [16]:
trainer.train()

Step,Training Loss,Validation Loss


{'eval_stsb_spearman': 0.6356538542097421, 'eval_sickr_spearman': 0.7364087253319732, 'eval_avg_sts': 0.6860312897708576}
MSRpar
MSRvid
SMTeuroparl
surprise.OnWN
surprise.SMTnews


C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


FNWN
headlines
OnWN
deft-forum
deft-news
headlines
images
OnWN
tweet-news
answers-forums
answers-students
belief
headlines
images
answer-answer
headlines
plagiarism
postediting
question-question
task STSBenchmark results[task] {'devpearson': 0.669761079244177, 'pearson': 0.6393349237026601, 'spearman': 0.6356538542097421, 'mse': 1.528943769716942, 'yhat': array([2.50391819, 1.94326281, 3.01332702, ..., 3.71976026, 4.12019033,
       4.64292709]), 'ndev': 1500, 'ntest': 1379}
task SICKRelatedness results[task] {'devpearson': 0.7920563030868204, 'pearson': 0.8155281336789253, 'spearman': 0.7364087253319732, 'mse': 0.3426460284378517, 'yhat': array([2.49770307, 3.80050344, 1.20151165, ..., 1.11200512, 3.06076166,
       4.33041312]), 'ndev': 500, 'ntest': 4927}
+-------+-------+-------+-------+-------+--------------+-----------------+-------+
| STS12 | STS13 | STS14 | STS15 | STS16 | STSBenchmark | SICKRelatedness |  Avg. |
+-------+-------+-------+-------+-------+--------------+---------

C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


...

FNWN
headlines
OnWN
deft-forum
deft-news
headlines
images
OnWN
tweet-news
answers-forums
answers-students
belief
headlines
images
answer-answer
headlines
plagiarism
postediting
question-question
task STSBenchmark results[task] {'devpearson': 0.7798566440940793, 'pearson': 0.7535703471604697, 'spearman': 0.7526480414533494, 'mse': 1.1203913110454276, 'yhat': array([1.43384198, 1.06934633, 2.7522268 , ..., 3.14906282, 3.51677624,
       4.01520283]), 'ndev': 1500, 'ntest': 1379}
task SICKRelatedness results[task] {'devpearson': 0.8389708828510081, 'pearson': 0.844080502823924, 'spearman': 0.7665785644195767, 'mse': 0.29672846426518185, 'yhat': array([3.3611607 , 3.99408721, 1.04631945, ..., 1.37263827, 3.05123463,
       4.61751782]), 'ndev': 500, 'ntest': 4927}
+-------+-------+-------+-------+-------+--------------+-----------------+-------+
| STS12 | STS13 | STS14 | STS15 | STS16 | STSBenchmark | SICKRelatedness |  Avg. |
+-------+-------+-------+-------+-------+--------------+-------

C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\JupyterDoc\SimCSE-reproduce-main/SentEval-main\senteval\sts.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


FNWN
headlines
OnWN
deft-forum
deft-news
headlines
images
OnWN
tweet-news
answers-forums
answers-students
belief
headlines
images
answer-answer
headlines
plagiarism
postediting
question-question
task STSBenchmark results[task] {'devpearson': 0.7819614668647625, 'pearson': 0.7649576556583645, 'spearman': 0.764007678811421, 'mse': 1.0602618562574242, 'yhat': array([1.57205723, 1.52914385, 2.60752142, ..., 4.32040943, 3.3866449 ,
       3.82319125]), 'ndev': 1500, 'ntest': 1379}
task SICKRelatedness results[task] {'devpearson': 0.8409532982032971, 'pearson': 0.8463136847315818, 'spearman': 0.7679873939125843, 'mse': 0.29155313952833717, 'yhat': array([3.65635441, 3.99775095, 1.10133373, ..., 1.29884879, 3.09823689,
       4.54710375]), 'ndev': 500, 'ntest': 4927}
+-------+-------+-------+-------+-------+--------------+-----------------+-------+
| STS12 | STS13 | STS14 | STS15 | STS16 | STSBenchmark | SICKRelatedness |  Avg. |
+-------+-------+-------+-------+-------+--------------+-------

{'eval_stsb_spearman': 0.7383297068420996, 'eval_sickr_spearman': 0.7638135739235594, 'eval_avg_sts': 0.7510716403828295}
{'eval_stsb_spearman': 0.7380046978416798, 'eval_sickr_spearman': 0.7634448827986906, 'eval_avg_sts': 0.7507247903201852}
{'eval_stsb_spearman': 0.7383778897511207, 'eval_sickr_spearman': 0.7639313091375848, 'eval_avg_sts': 0.7511545994443527}
{'eval_stsb_spearman': 0.7396529844428529, 'eval_sickr_spearman': 0.7635169744341057, 'eval_avg_sts': 0.7515849794384792}
{'eval_stsb_spearman': 0.7400251913446453, 'eval_sickr_spearman': 0.7638469881663659, 'eval_avg_sts': 0.7519360897555056}
{'eval_stsb_spearman': 0.7403129120787797, 'eval_sickr_spearman': 0.7633072736591682, 'eval_avg_sts': 0.7518100928689739}
{'eval_stsb_spearman': 0.743170904898831, 'eval_sickr_spearman': 0.7681249301723967, 'eval_avg_sts': 0.7556479175356139}
{'eval_stsb_spearman': 0.7419033168816669, 'eval_sickr_spearman': 0.7676830564249384, 'eval_avg_sts': 0.7547931866533026}
{'eval_stsb_spearman': 0.

TrainOutput(global_step=15554, training_loss=0.0002497303786418587, metrics={'train_runtime': 17679.5594, 'train_samples_per_second': 56.305, 'train_steps_per_second': 0.88, 'total_flos': 3.296430570287923e+16, 'train_loss': 0.0002497303786418587, 'epoch': 1.0})